##### 株価時系列の可視化####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import matplotlib
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
from matplotlib.backends.backend_pdf import PdfPages 
import sys

#np.random.seed(98537)

In [ ]:
##データの読み込み
panel_data = pd.read_csv("D:/Statistics/data/stock_price/stock_panel_data.csv", index_col=0)
company_info = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv")
date_range = [np.min(panel_data["日付"]), np.max(panel_data["日付"])]

In [ ]:
#株価の推移をプロット
i = 0
company_info = company_info.iloc[np.array(company_info["業種"]!="ETF")]
company_info.index = np.arange(company_info.shape[0])
codes = np.array(company_info["コード"])
price = panel_data[["コード", "銘柄名", "日付", "終値"]].iloc[np.array(panel_data[["コード"]]==codes[i]).reshape(-1)]
price[["日付"]] = pd.to_datetime(price["日付"])

df = price.set_index("日付")
legend = str(np.array(price[["銘柄名"]].iloc[0:1]).reshape(-1) + "の株価推移"); legend = legend[2:len(legend)-2]
ax = plt.subplot()
ax.plot(df.index, df["終値"])
ax.set_xlim(date_range[0], date_range[1])
plt.title(legend)
plt.show()

In [ ]:
##業界のすべての株価推移をプロット
def multi_plot(panel_data, industry, industry_eng, display_n=6, displany_n0=[4, 2]):
    #データの抽出と設定
    index_sub = np.array(np.where(np.array(panel_data["業種"])==industry)[0], dtype="int")
    sub_data = panel_data.iloc[index_sub]
    company_name = np.unique(np.array(sub_data["コード"]))
    n = company_name.shape[0]

    #出力ファイルごとのインデックスを設定
    if n < display_n:
        display_n = n
    out = "D:\Statistics\data/image/{industry_eng}_stock_price{no}.jpg"
    out_n = int(np.round(n / display_n))
    max_n = -1
    index_list = [i for i in range(out_n)]
    for i in range(out_n):    
        index_list[i] = np.arange(n)[max_n+1:max_n+display_n+1]
        index_list[i] = index_list[i][index_list[i] < n]
        max_n = np.max(index_list[i])

    #プロットの設定
    for i in range(out_n):
        get_company = company_name[index_list[i]]
        colorlist = ["r", "g", "b", "c", "m", "y", "k", "w"]
        fig = plt.figure(figsize=(20.0, 20.0), dpi=50)

        for j in range(get_company.shape[0]):
            #プロットするデータの設定
            price = sub_data[["コード", "銘柄名", "日付", "終値"]].iloc[np.array(sub_data["コード"]==get_company[j])]
            price[["日付"]] = pd.to_datetime(price["日付"])
            df = price.set_index("日付")
            legend = str(np.array(price[["銘柄名"]].iloc[0:1]).reshape(-1) + "の株価推移"); legend = legend[2:len(legend)-2]

            #グラフを描画
            plt.subplots_adjust(wspace=0.4, hspace=0.65)   #余白を設定
            ax = plt.subplot(display_n0[0], display_n0[1], j+1)
            ax.plot(df.index, df["終値"])
            ax.set_xlim(date_range[0], date_range[1])
            plt.setp(ax.get_yticklabels(), fontsize=12.5)
            plt.setp(ax.get_xticklabels(), rotation=40, fontsize=12.5)
            plt.title(legend, fontsize=20)
        fig.tight_layout()
        plt.savefig(out.format(industry_eng=industry_eng, no=i))

In [ ]:
#すべての業界の株価推移をすべてプロット
print(pd.unique(panel_data["業種"]))
industry_list = pd.unique(panel_data["業種"])
display_n = 10
display_n0 = [5, 2]
for i in range(len(industry_list)):
    print(industry_list[i])
    multi_plot(panel_data, industry_list[i], industry_list[i], display_n=10, displany_n0=[5, 2])